In [1]:
import pandas as pd
import numpy as np
import warnings
from math import sqrt
warnings.filterwarnings('ignore')
from azureml.core.run import Run
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.model import Model
from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.train.automl import AutoMLConfig
import pickle
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import mlflow

Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (packaging 23.0 (/Users/krishnayogi/ls/envs/myenv_x86/lib/python3.8/site-packages), Requirement.parse('packaging<22.0,>=20.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (packaging 23.0 (/Users/krishnayogi/ls/envs/myenv_x86/lib/python3.8/site-packages), Requirement.parse('packaging<22.0,>=20.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (packaging 23.0 (/Users/krishnayogi/ls/envs/myenv_x86/lib/python3.8/site-packages), Requirement.parse('packaging<22.0,>=20.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun

In [2]:
from azureml.core import Workspace, Dataset

subscription_id = '3566c1ea-5752-413c-9cf4-e7e55fca7d8f'
resource_group = 'dp100'
workspace_name = 'aml'

workspace = Workspace(subscription_id, resource_group, workspace_name)

In [3]:
uri = workspace.get_mlflow_tracking_uri()
mlflow.set_tracking_uri(uri)

In [4]:
print(uri)

azureml://southeastasia.api.azureml.ms/mlflow/v2.0/subscriptions/3566c1ea-5752-413c-9cf4-e7e55fca7d8f/resourceGroups/dp100/providers/Microsoft.MachineLearningServices/workspaces/aml?


In [5]:
datastore = workspace.get_default_datastore()
print(datastore)

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-6d61d964-6aca-41cd-a070-05ade70b8520",
  "account_name": "aml0148788539",
  "protocol": "https",
  "endpoint": "core.windows.net"
}


In [8]:
# Importing pre-processed dataset
dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, 'data/weather_dataset_processed.csv')])

#dataset = Dataset.get_by_name(workspace, name='data/weather_dataset_processed.csv')
print(dataset.name, dataset.version)

None None


In [10]:
print(dataset)

TabularDataset
{
  "source": [
    "('workspaceblobstore', 'data/weather_dataset_processed.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}


In [11]:
df = dataset.to_pandas_dataframe()

In [12]:
df.head()

,Timestamp,Location,Temperature_C,Humidity,Wind_speed_kmph,Wind_bearing_degrees,Visibility_km,Pressure_millibars,Weather_conditions
0,2006-03-31 22:00:00,"Port of Turku, Finland",9.472222,0.89,14.1197,251,15.8263,1015.13,rain
1,2006-03-31 23:00:00,"Port of Turku, Finland",9.355556,0.86,14.2646,259,15.8263,1015.63,rain
2,2006-04-01 00:00:00,"Port of Turku, Finland",9.377778,0.89,3.9284,204,14.9569,1015.94,rain
3,2006-04-01 01:00:00,"Port of Turku, Finland",8.288889,0.83,14.1036,269,15.8263,1016.41,rain
4,2006-04-01 02:00:00,"Port of Turku, Finland",8.755556,0.83,11.0446,259,15.8263,1016.51,rain


# Spliting Pre-Processed data into Training and Validation datasets

In [13]:
# Validation set is used later to evaluate model performance post training. 

In [14]:
df_training = df.iloc[:77160]

In [15]:
df_training.shape

(77160, 9)

In [16]:
df_validation = df.drop(df_training.index)

In [17]:
df_validation.shape

(19293, 9)

# Registering Training and Validation data to the datastore on the workspace. 

In [18]:
!mkdir Data

mkdir: Data: File exists


In [19]:
df_training.to_csv('Data/training_data.csv',index=False)

In [20]:
df_validation.to_csv('Data/validation_data.csv',index=False)

In [21]:
datastore = workspace.get_default_datastore()

In [22]:
datastore.upload(src_dir='Data', target_path='data')

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 2 files
Target already exists. Skipping upload for data/training_data.csv
Target already exists. Skipping upload for data/validation_data.csv
Uploaded 0 files


$AZUREML_DATAREFERENCE_7dea0446f077421e809b31a81dfddbb3

In [23]:
training_dataset = Dataset.Tabular.from_delimited_files(datastore.path('data/training_data.csv'))

In [24]:
validation_dataset = Dataset.Tabular.from_delimited_files(datastore.path('data/validation_data.csv'))

In [25]:
training_ds = training_dataset.register(workspace=workspace,
                                 name='training_dataset',
                                 description='Dataset to use for ML training')

In [26]:
validation_ds = validation_dataset.register(workspace=workspace,
                                 name='validation_dataset',
                                 description='Dataset for validation ML models')

# Data ingestion step - Training dataset

In [27]:
dataset = Dataset.get_by_name(workspace, name='training_dataset')
print(dataset.name, dataset.version)

training_dataset 1


In [28]:
df = dataset.to_pandas_dataframe()

In [29]:
df.head()

,Timestamp,Location,Temperature_C,Humidity,Wind_speed_kmph,Wind_bearing_degrees,Visibility_km,Pressure_millibars,Weather_conditions
0,2006-03-31 22:00:00,"Port of Turku, Finland",9.472222,0.89,14.1197,251,15.8263,1015.13,rain
1,2006-03-31 23:00:00,"Port of Turku, Finland",9.355556,0.86,14.2646,259,15.8263,1015.63,rain
2,2006-04-01 00:00:00,"Port of Turku, Finland",9.377778,0.89,3.9284,204,14.9569,1015.94,rain
3,2006-04-01 01:00:00,"Port of Turku, Finland",8.288889,0.83,14.1036,269,15.8263,1016.41,rain
4,2006-04-01 02:00:00,"Port of Turku, Finland",8.755556,0.83,11.0446,259,15.8263,1016.51,rain


In [30]:
df.shape

(77160, 9)

#### Feature Selection and scaling

In [32]:
X = df[['Temperature_C', 'Humidity', 'Wind_speed_kmph', 'Wind_bearing_degrees', 'Visibility_km', 'Pressure_millibars']].values
y = df['Weather_conditions'].values
y

array(['rain', 'rain', 'rain', ..., 'rain', 'rain', 'rain'], dtype=object)

In [33]:
# Splitting the Training dataset into Train and Test set for ML training
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

ImportError: cannot import name 'StandardScaler' from 'sklearn.preprocessing' (unknown location)

In [35]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Model training and Testing Step

## 1. Support Vector Machine

In [36]:
myexperiment = Experiment(workspace, "support-vector-machine")
mlflow.set_experiment("mlflow-support-vector-machine")

2023/01/17 17:02:05 INFO mlflow.tracking.fluent: Experiment with name 'mlflow-support-vector-machine' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1673955125857, experiment_id='34f93cbc-9288-4731-b22a-59b53ef3664b', last_update_time=None, lifecycle_stage='active', name='mlflow-support-vector-machine', tags={}>

In [37]:
#from sklearn.svm import SVC
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

In [38]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

In [39]:
svc = svm.SVC()

In [40]:
print(dataset.name,dataset.version)

training_dataset 1


In [41]:
# initialize a run in Azureml and mlflow experiments
run = myexperiment.start_logging()
mlflow.start_run()


run.log("dataset name", dataset.name)
run.log("dataset Version", dataset.version)

In [42]:
svc_grid = GridSearchCV(svc, parameters)

In [43]:
%%time
svc_grid.fit(X_train, y_train)

CPU times: user 1min 13s, sys: 1.41 s, total: 1min 14s
Wall time: 1min 16s


GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})

In [44]:
svc_grid.get_params(deep=True)

{'cv': None,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__break_ties': False,
 'estimator__cache_size': 200,
 'estimator__class_weight': None,
 'estimator__coef0': 0.0,
 'estimator__decision_function_shape': 'ovr',
 'estimator__degree': 3,
 'estimator__gamma': 'scale',
 'estimator__kernel': 'rbf',
 'estimator__max_iter': -1,
 'estimator__probability': False,
 'estimator__random_state': None,
 'estimator__shrinking': True,
 'estimator__tol': 0.001,
 'estimator__verbose': False,
 'estimator': SVC(),
 'n_jobs': None,
 'param_grid': {'kernel': ('linear', 'rbf'), 'C': [1, 10]},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': None,
 'verbose': 0}

In [45]:
from sklearn.svm import SVC

In [46]:
svc = SVC(C=svc_grid.get_params(deep=True)['estimator__C'], kernel=svc_grid.get_params(deep=True)['estimator__kernel'])

In [47]:
svc.fit(X_train, y_train)

SVC()

In [48]:
# Logging training parameters to AzureML and MLFlow experiments
run.log("C", svc_grid.get_params(deep=True)['estimator__C'])
run.log("Kernel", svc_grid.get_params(deep=True)['estimator__kernel'])

In [49]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [50]:
predicted_svc = svc.predict(X_test)

In [51]:
acc = accuracy_score(y_test, predicted_svc)

In [52]:
fscore = f1_score(y_test, predicted_svc, average="macro")
precision = precision_score(y_test, predicted_svc, average="macro")
recall = recall_score(y_test, predicted_svc, average="macro")

In [53]:
import git
repo = git.Repo(search_parent_directories=True)
sha = repo.head.object.hexsha

In [55]:
print(repo, sha)

<git.repo.base.Repo '/Users/krishnayogi/myenv_x86/EngineeringMLOps/.git'> b8de9f7e85131e5784b9275da413462a4c750a8b


In [56]:
# Log to AzureML and MLflow
run.log("Test_accuracy", acc)
run.log("Precision", precision)
run.log("Recall", recall)
run.log("F-Score", fscore)
run.log("Git-sha", sha)

In [57]:
run.complete()
print ("run id:", run.id)

run id: 4b7a7286-cc74-4c14-8e4f-a7f8934732b5


In [58]:
mlflow.end_run()

In [59]:
run.get_metrics()

{'dataset name': 'training_dataset',
 'dataset Version': 1,
 'C': 1.0,
 'Kernel': 'rbf',
 'Test_accuracy': 0.9916407465007776,
 'Precision': 0.9736206238419904,
 'Recall': 0.987824315716418,
 'F-Score': 0.9805771293084771,
 'Git-sha': 'b8de9f7e85131e5784b9275da413462a4c750a8b'}

In [60]:
workspace.get_details()

{'id': '/subscriptions/3566c1ea-5752-413c-9cf4-e7e55fca7d8f/resourceGroups/dp100/providers/Microsoft.MachineLearningServices/workspaces/aml',
 'name': 'aml',
 'identity': {'principal_id': '20706030-2cbf-4a0e-8263-49c779061f9f',
  'tenant_id': 'f2ef5e7d-a841-46c8-85c1-056389d5dda3',
  'type': 'SystemAssigned'},
 'location': 'southeastasia',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': '6d61d964-6aca-41cd-a070-05ade70b8520',
 'sdkTelemetryAppInsightsKey': 'c59896f9-68fe-4b35-bb22-2f0e798a3858',
 'description': '',
 'friendlyName': 'aml',
 'containerRegistry': '/subscriptions/3566c1ea-5752-413c-9cf4-e7e55fca7d8f/resourceGroups/dp100/providers/Microsoft.ContainerRegistry/registries/6d61d9646aca41cda07005ade70b8520',
 'keyVault': '/subscriptions/3566c1ea-5752-413c-9cf4-e7e55fca7d8f/resourceGroups/dp100/providers/Microsoft.Keyvault/vaults/aml4325789331',
 'applicationInsights': '/subscriptions/3566c1ea-5752-413c-9cf4-e7e55fca7d8f/reso

In [61]:
import mlflow.sklearn
mlflow.sklearn.log_model(svc, 'outputs')

2023/01/17 17:53:01 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/_x/15zkvlz965jbcyj2tvmx0ww80000gn/T/tmpc5yopagy/model/model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.2.0', 'cloudpickle==2.2.0']. Set logging level to DEBUG to see the full traceback.


Random Forest classifier 

In [62]:
myexperiment = Experiment(workspace, "random-forest-classifier")
mlflow.set_experiment("mlflow-random-forest-classifier")

2023/01/17 17:53:11 INFO mlflow.tracking.fluent: Experiment with name 'mlflow-random-forest-classifier' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1673958192313, experiment_id='853e38c0-70c1-47e1-bc0e-de2c48b86200', last_update_time=None, lifecycle_stage='active', name='mlflow-random-forest-classifier', tags={}>

In [63]:
from sklearn.ensemble import RandomForestClassifier

In [64]:
rf = RandomForestClassifier(max_depth=10, random_state=0, n_estimators=100)

In [65]:
# initialize runs in Azureml and mlflow
run = myexperiment.start_logging()
mlflow.start_run()


# Log dataset used 
run.log("dataset name", dataset.name)
run.log("dataset Version", dataset.version)

Exception: Run with UUID 09a9fee8-b710-4be0-ace5-69392c4248a6 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True

In [66]:
%%time
rf.fit(X_train, y_train)

CPU times: user 1.78 s, sys: 29 ms, total: 1.81 s
Wall time: 1.87 s


RandomForestClassifier(max_depth=10, random_state=0)

In [92]:
import pickle
# now you can save it to a file
with open('outputs/rf.pkl', 'wb') as f:
    pickle.dump(rf, f)


    
# saving  svc

with open('outputs/svc.pkl', 'wb') as f:
    pickle.dump(svc, f)



In [3]:
!pwd
#from sklearn.preprocessing import StandardScaler
#!conda update --all -y

/Users/krishnayogi/myenv_x86/EngineeringMLOps/04_ML_Pipelines


In [4]:
import pickle
# and later you can load it
with open('outputs/rf.pkl', 'rb') as f:
    rf = pickle.load(f)
    
# and later you can load it
with open('outputs/svc.pkl', 'rb') as f:
    svc = pickle.load(f)

ModuleNotFoundError: No module named 'sklearn'

In [6]:
#!python3 -m pip install scikit-learn
#!pip3 install --user scikit-learn

  Using cached scikit_learn-1.2.0-cp38-cp38-macosx_10_9_x86_64.whl (8.9 MB)
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/Users/krishnayogi/ls/envs/myenv_x86/lib/python3.8/site-packages/numpy-1.24.1.dist-info/METADATA'



In [91]:
!ls

Data              outputs           svc.pkl
ML-pipeline.ipynb rf.pkl


In [67]:
# Logging training parameters to AzureML and MLFlow experiments
run.log("max_depth", 10)
run.log("random_state", 0)
run.log("n_estimators", 100)

In [68]:
predicted_rf = rf.predict(X_test)

In [69]:
acc = accuracy_score(y_test, predicted_rf)
fscore = f1_score(y_test, predicted_rf, average="macro")
precision = precision_score(y_test, predicted_rf, average="macro")
recall = recall_score(y_test, predicted_rf, average="macro")

In [70]:
run.log("Test_accuracy", acc)
run.log("Precision", precision)
run.log("Recall", recall)
run.log("F-Score", fscore)
run.log("Git-sha", sha)

In [71]:
run.complete()
print ("run id:", run.id)

run id: 8ae01f8c-88c1-430e-b88b-0d1b466c8522


In [72]:
mlflow.end_run()

In [73]:
run.get_metrics()

{'max_depth': 10,
 'random_state': 0,
 'n_estimators': 100,
 'Test_accuracy': 1.0,
 'Precision': 1.0,
 'Recall': 1.0,
 'F-Score': 1.0,
 'Git-sha': 'b8de9f7e85131e5784b9275da413462a4c750a8b'}

In [86]:
import numpy
numpy.version.version

'1.23.5'

In [83]:
#!conda install -c conda-forge skl2onnx -y


import onnx
import numpy
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
#from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from onnxruntime import InferenceSession
from skl2onnx import convert_sklearn, to_onnx, wrap_as_onnx_mixin
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx.algebra.onnx_ops import OnnxSub, OnnxDiv
from skl2onnx.algebra.onnx_operator_mixin import OnnxOperatorMixin

ImportError: cannot import name '_is_pairwise' from 'sklearn.base' (/Users/krishnayogi/ls/envs/myenv_x86/lib/python3.8/site-packages/sklearn/base.py)

In [ ]:
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

# Model Packaging Step

pickle file or onnx

In [98]:
# Convert into SVC model into ONNX format file
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([None, 6]))]
onx = convert_sklearn(svc, initial_types=initial_type)
with open("outputs/svc.onnx", "wb") as f:
    f.write(onx.SerializeToString())

ModuleNotFoundError: No module named 'sklearn.gaussian_process'

In [ ]:
# Convert into RF model into ONNX format file
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([None, 6]))]
onx = convert_sklearn(rf, initial_types=initial_type)
with open("outputs/rf.onnx", "wb") as f:
    f.write(onx.SerializeToString())

# Model Registering Step

In [93]:
# Register Model on AzureML WS
model = Model.register(model_path = './outputs/svc.pkl', # this points to a local file  # svc.onnx normally
                       model_name = "support-vector-classifier", # this is the name the model is registered as
                       tags = {'dataset': dataset.name, 'version': dataset.version, 'hyparameter-C': '1', 'testdata-accuracy': '0.9519'}, 
                       model_framework='pandas==0.23.4',
                       description = "Support vector classifier to predict weather at port of Turku",
                       workspace = workspace)

print('Name:', model.name)
print('Version:', model.version)

Registering model support-vector-classifier
Name: support-vector-classifier
Version: 1


In [94]:
# Register Model on AzureML WS
model = Model.register(model_path = './outputs/rf.pkl', # this points to a local file  # rf.onnx normally
                       model_name = "random-forest-classifier", # this is the name the model is registered as
                       tags = {'dataset': dataset.name, 'version': dataset.version, 'hyparameter-C': '1', 'testdata-accuracy': '0.9548'}, 
                       model_framework='pandas==0.23.4',
                       description = "Random forest classifier to predict weather at port of Turku",
                       workspace = workspace)

print('Name:', model.name)
print('Version:', model.version)

Registering model random-forest-classifier
Name: random-forest-classifier
Version: 1


In [95]:
import mlflow.sklearn

In [96]:
# Save the model to the outputs directory for capture
#mlflow.sklearn.log_model(svc, 'outputs/svc.onnx')

mlflow.sklearn.log_model(svc, 'outputs/svc.pkl')

2023/01/17 19:00:27 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/_x/15zkvlz965jbcyj2tvmx0ww80000gn/T/tmpqngco8sc/model/model.pkl, flavor: sklearn), fall back to return ['scikit-learn==0.24.2', 'cloudpickle==2.2.0']. Set logging level to DEBUG to see the full traceback.


In [97]:
# Save the model to the outputs directory for capture
#mlflow.sklearn.log_model(rf, 'outputs/rf.onnx')
mlflow.sklearn.log_model(rf, 'outputs/rf.pkl')

2023/01/17 19:00:50 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/_x/15zkvlz965jbcyj2tvmx0ww80000gn/T/tmpa7_74opb/model/model.pkl, flavor: sklearn), fall back to return ['scikit-learn==0.24.2', 'cloudpickle==2.2.0']. Set logging level to DEBUG to see the full traceback.


# Save model artefacts

In [ ]:
import pickle

with open('./outputs/scaler.pkl', 'wb') as scaler_pkl:
    pickle.dump(sc, scaler_pkl)

In [ ]:
# Register Model on AzureML WS
scaler = Model.register(model_path = './outputs/scaler.pkl', # this points to a local file 
                       model_name = "scaler", # this is the name the model is registered as
                       tags = {'dataset': dataset.name, 'version': dataset.version}, 
                       model_framework='pandas==0.23.4',
                       description = "Scaler used for scaling incoming inference data",
                       workspace = workspace)

print('Name:', scaler.name)
print('Version:', scaler.version)